In [517]:
import pymysql as ms
import pandas as pd

#Establishing Connection to MySQL
connection = ms.connect(host='127.0.0.1',
                        user = 'root',
                        password = 'PassWord',
                        db = 'project',
                        cursorclass = ms.cursors.DictCursor)
cursor = connection.cursor()

OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")

In [523]:
#Select columns to use
factor = pd.read_csv("number-of-deaths-by-risk-factor.csv", usecols = [0, 2, 23, 6, 29], names = ["Location_ID","Year", "Smoking_Deaths", "Household_Air Pollution_Deaths", "Outdoor Air Pollution Deaths"], skiprows = 1)
delete = factor[(factor["Year"] ==1990)|(factor["Year"]== 1991)|(factor["Year"]== 1992)|(factor["Year"]== 1993)|(factor["Year"]== 1994)|(factor["Year"]== 1995)|(factor["Year"]== 1996)|(factor["Year"]== 1997)|(factor["Year"]== 1998)|(factor["Year"]== 1999)|(factor["Year"]== 2000)|(factor["Year"]== 2001)|(factor["Year"]== 2002)|(factor["Year"]== 2003)|(factor["Year"]== 2004)|(factor["Year"]== 2005)|(factor["Year"]== 2006)|(factor["Year"]== 2007)|(factor["Year"]== 2008)|(factor["Year"]== 2009)|(factor["Year"]== 2010)|(factor["Year"]== 2011)|(factor["Year"]== 2012)|(factor["Year"]== 2013)|(factor["Year"]== 2014)|(factor["Year"]== 2015)|(factor["Year"]== 2016)].index
factor.drop(delete , inplace = True)

PM25 = pd.read_csv("PM25.csv", usecols = [0, 7, 8, 9], names = ["Location_ID", "PM2015", "PM2016", "PM2017"], skiprows = 1)
covid_case = pd.read_csv("COVID_Case.csv",low_memory=False, usecols = [0,1, 2,7] ,names =["Location_ID","Region","cumulative_cases",  "cumulative_death"])



In [524]:
#Innerjoin to create fact table dataset
fact = pd.merge(factor,PM25, how = "inner", on = "Location_ID")
joined = fact[["Location_ID","PM2015", "PM2016", "PM2017", "Smoking_Deaths", "Household_Air Pollution_Deaths", "Outdoor Air Pollution Deaths"]]
fact_table = pd.merge(joined, covid_case, how = "inner", on = "Location_ID")

Correlation_Fact = pd.merge(right = fact_table, left = covid_case, how = "inner")
Correlation_Fact = Correlation_Fact[["Location_ID","PM2015", "PM2016", "PM2017", "Smoking_Deaths","Household_Air Pollution_Deaths", "Outdoor Air Pollution Deaths"]]



In [525]:
#Innerjoin to create location_dim dataset
location_dim = pd.merge(right = fact, left = covid_case, how = "inner", on = "Location_ID")
location_dim = location_dim[["Location_ID","Region"]]


In [526]:
#Create location_dim table
sql2 = """CREATE TABLE IF NOT EXISTS Location_Dim (
            Location_ID varchar(100), 
            Country varchar(100),
            PRIMARY KEY (Location_ID)
            );"""

cursor.execute(sql2)
connection.commit()

location_dim_data = [tuple(m) for m in location_dim.values.tolist()]

cursor.execute("TRUNCATE TABLE Location_Dim;")
connection.commit()

#Insert data into table
insertData = "INSERT INTO Location_Dim VALUES (%s, %s);"
cursor.executemany(insertData, location_dim_data)
connection.commit()


In [527]:
#Create Correlation_Fact table
sql = """CREATE TABLE IF NOT EXISTS Correlation_Fact (
            Location_ID varchar(100), 
            PM2015 varchar(1000), 
            PM2016 varchar(1000), 
            PM2017 varchar(1000), 
            Smoking_Deaths varchar(1000), 
            Household_Air_Pollution_Deaths varchar(1000), 
            Outdoor_Air_Pollution_Deaths varchar(1000),
            FOREIGN KEY (Location_ID) REFERENCES Location_Dim(Location_ID)
            );"""


cursor.execute(sql)
connection.commit()

Correlation_Fact_data = [tuple(i) for i in Correlation_Fact.values.tolist()]
cursor.execute("TRUNCATE TABLE Correlation_Fact;")
connection.commit()

#Insert data into table
insertData_fact = "INSERT INTO Correlation_Fact VALUES (%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(insertData_fact, Correlation_Fact_data)
connection.commit()